<a href="https://colab.research.google.com/gist/hnatar/ca03aca0175ca763811c1e73a9ccd8d8/lstm-text-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

# Data had e-mails so I've removed the files. Please contact Bo
# if you want the original notebook with outputs.
class1 = 'foo.txt' # Entries from one field, newline delimited
class2 = 'bar.txt' # Entries from other field, newline delimited

"""
Training took a while on Colab, so limited to 30.
"""
def load_from_file(src):
    i = 0
    samples = []
    with open(f'{src}', 'r') as f:
        for line in f.readlines():
            line = line.strip()
            samples.append( line )
            i += 1
            if i == 30:
                break
    return samples

"""
Samples is a dictionary holding the input
samples for that field/class. There are two classes, 1 and 2.
"""
samples = {}
samples[1] = load_from_file(class1)
samples[2] = load_from_file(class2)

"""
Please refactor this out.
"""
def text2ascii_onehot(s):
    """ returns a (n x 255) tensor where n is len(s),
        containing one-hot ascii encoding for s
    """
    x = []
    for c in s:
        row = [0]*256
        row[ord(c)] = 1
        x.append( np.array(row) )
    return np.array(x)

"""
Generates a  sample, ie. if the two inputs are similar/from the
same class: pick a class randomly, and pick two random samples
from that class. Also returns the ground truth labelling.
"""
def generate_valid():
    r = np.random.randint(1, 3)
    textSample1 = samples[r][ np.random.randint(0, len(samples[r])) ]
    textSample2 = samples[r][ np.random.randint(0, len(samples[r])) ]
    x1 = text2ascii_onehot(textSample1)
    x2 = text2ascii_onehot(textSample2)
    return x1, x2, np.array([0, 1]), (textSample1, textSample2,)

def generate_invalid():
    textSample1 = samples[1][ np.random.randint(0, len(samples[1])) ]
    textSample2 = samples[2][ np.random.randint(0, len(samples[2])) ]
    r = np.random.randint(1, 3)
    x1 = text2ascii_onehot(textSample1)
    x2 = text2ascii_onehot(textSample2)
    if r == 1:
        return x1, x2, np.array([1, 0]), (textSample1, textSample2,)
    else:
        return x2, x1, np.array([1, 0]), (textSample2, textSample1,)


In [0]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Bidirectional, Dot, Concatenate, Subtract, Average
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras import backend as K
from keras import regularizers
import numpy as np

text1 = Input(shape=(None, 256))
text2 = Input(shape=(None, 256))
# concat and return_sequences to retain info
shared_blstm1 = Bidirectional( LSTM(units=4, input_shape=(None, 256), return_sequences=True), merge_mode='concat')
shared_blstm2 = Bidirectional( LSTM(units=4, input_shape=(None, 256)), merge_mode='concat')
transform1 = Dense(128)
embedding1 = transform1(shared_blstm2(shared_blstm1(text1)))
embedding2 = transform1(shared_blstm2(shared_blstm1(text2)))

#sim_layer = Dense(units=8)
similarity = Dot(axes=-1, normalize=False)([embedding1, embedding2])
softmax_layer = Dense(2, activation='softmax')
predict = softmax_layer(similarity)
model = Model(inputs=[text1, text2], outputs=predict)

# model = Sequential()
# model.add(  )
# model.add( Dense(4) )
# model.add( Dense(4) )
# model.add( Dense(2, activation='softmax') )
# model_opt = SGD(lr=0.01, momentum=0.3, decay=0, nesterov=False)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

def get_random_samples(size):
    for i in range(size):
        r = np.random.randint(1,3)
        if r == 1:
            return generate_valid()
        else:
            return generate_invalid()

In [0]:
print( model.summary() )

for layer in model.layers:
    print("Input shape: "+str(layer.input_shape)+". Output shape: "+str(layer.output_shape))
    
for epoch in range(100):
    # perform some training
    for _ in range(100):
        x1, x2, label, data = get_random_samples(1)
        x1 = np.array([x1])
        x2 = np.array([x2])
        label = label.reshape(1, 2)
        model.fit([x1,x2], label, verbose=False)
    # check results
    x1, x2, label, data = get_random_samples(1)
    x1 = np.array([x1])
    x2 = np.array([x2])
    label = label.reshape(1, 2)
    r = model.predict([x1,x2])
    p1, p2 = np.exp(r[0][0]), np.exp(r[0][1])
    p1, p2 = p1/(p1+p2), p2/(p1+p2)
    p1 *= 100.0
    p2 *= 100.0
    
    predicted_label = model.predict([x1,x2]).argmax(axis=-1)
    print(data, predicted_label, p1 , p2)
    print(score)


# for i in range(10):
#     x, y, samples = get_random_samples(1)
#     predicted_label = model.predict(x).argmax(axis=-1)
#     print( samples, y, predicted_label )